# 37. PYT. Продвинутые приёмы работы с данными в Pandas

В первой части мы поговорим о продвинутых методах Pandas, позволяющих производить более детальный анализ зависимостей в данных и делать на их основе выводы. 

Вторую часть посвятим работе с несколькими таблицами и их объединению.

In [ ]:
import pandas as pd

melb_df = pd.read_csv('data/melb_data_fe.csv')
display(melb_df.info())
melb_df.head()

Условимся, что категориальными будут считаться признаки, у которых число уникальных категорий меньше 150. При этом к категориальным не будут относиться следующие столбцы: Date, Rooms, Bedroom, Bathroom, Car.

Преобразуйте выделенные столбцы (Date, Rooms, Bedroom, Bathroom, Car) в тип данных category.
В качестве ответа запишите результирующее количество столбцов, которые имеют тип данных category.

In [ ]:
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца
display(melb_df.info())

Преобразуйте столбец Date в формат datetime и выделите квартал (quarter) продажи объектов недвижимости. Найдите второй по популярности квартал продажи.
В качестве ответа запишите число объектов, проданных в этом квартале.

In [59]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
melb_df['Quarter'] = melb_df['Date'].dt.quarter
melb_df['Quarter'].value_counts()
melb_df[melb_df['Quarter']  == 2]['Price'].count()

4359

In [ ]:
melb_df['Quarter'] = melb_df['Date'].dt.quarter
melb_df['Quarter'].value_counts()
melb_df[melb_df['Quarter']  == 2]['Price'].count()


4359

## 37.2 Сортировка данных в DataFrame

### Основные параметры метода sort_values()

→ by — задаёт имя или список имён столбцов, по значениям которых производится сортировка.

→ axis — задаёт ось, по которой производится сортировка (0 — строки, 1 — столбцы). По умолчанию сортировка производится по строкам.

→ ascending — задаёт сортировку по возрастанию (от меньшего к большему): по умолчанию параметр выставлен на True. Для сортировки по убыванию (от большего к меньшему) необходимо изменить значение на False.

→ ignore_index — задаёт создание новых индексов в таблице. По умолчанию выставлен на False и сохраняет индексы изначальной таблицы.

→ inplace — задаёт замену исходной таблицы на отсортированную. По умолчанию параметр выставлен на False, то есть замены не производится. Чтобы переопределить исходную таблицу на отсортированную, необходимо выставить этот параметр на True.

In [ ]:
# СОРТИРОВКА ПО ЗНАЧЕНИЯМ ОДНОГО СТОЛБЦА

melb_df.sort_values(by='Price').head(10)
melb_df.sort_values(by='Date', ascending=False)

In [ ]:
# СОРТИРОВКА ПО ЗНАЧЕНИЯМ НЕСКОЛЬКИХ СТОЛБЦОВ

"""Отсортируем таблицу сначала по возрастанию расстояния от центра города (Distance), 
а затем — по возрастанию цены объекта (Price).
Чтобы вывод был более наглядным, выделим каждую десятую строку из столбцов Distance 
и Price результирующей таблицы."""

melb_df.sort_values(by=['Distance', 'Price']).loc[::10, ['Distance', 'Price']]

In [ ]:
melb_df.sort_values(by=['Price', 'Distance']).loc[::10, ['Distance', 'Price']]

In [ ]:
# КОМБИНИРОВАНИЕ СОРТИРОВКИ С ФИЛЬТРАЦИЕЙ

""" Найдём информацию о таунхаусах (Type), проданных компанией (SellerG) McGrath, 
у которых коэффициент соотношения площадей здания и участка (AreaRatio) меньше -0.8. 
Результат отсортируем по дате продажи (Date) в порядке возрастания, а после проведём 
сортировку по убыванию коэффициента соотношения площадей. Также обновим старые 
индексы на новые, установив параметр ignore_index на True.
Для наглядности результата выберем из таблицы только столбцы SellerG и AreaRatio."""

mask1 = melb_df['AreaRatio'] < -0.8
mask2 = melb_df['Type'] == 'townhouse'
mask3 = melb_df['SellerG'] == 'McGrath'
melb_df[mask1 & mask2 & mask3].sort_values(
    by=['Date', 'AreaRatio'],
    ascending=[True, False],
    ignore_index=True
).loc[:, ['Date', 'AreaRatio']]

In [35]:
"""Задание 37.2.2 Произведите сортировку столбца AreaRatio по убыванию. 
При этом индексы полученной таблицы замените на новые.
Какое значение площади здания находится в строке 1558?"""

int(melb_df.sort_values(by='AreaRatio', ignore_index=True, ascending=False).loc[1558, 'BuildingArea'])

126

In [34]:
melb_df['Type'].value_counts()

house        9449
unit         3017
townhouse    1114
Name: Type, dtype: int64

In [36]:
"""Найдите таунхаусы (соответствующее значение в столбце Type) с количеством жилых 
комнат (Rooms) больше двух. Отсортируйте полученную таблицу сначала по возрастанию 
числа комнат, а затем по убыванию средней площади комнат (MeanRoomsSquare). Индексы 
таблицы замените на новые. Какая цена будет у объекта в строке 18?"""

mask1 = melb_df['Rooms'] > 2
mask2 = melb_df['Type'] == 'townhouse'
melb_df[mask1 & mask2].sort_values(
    by=['Rooms', 'MeanRoomsSquare'],
    ascending=[True, False],
    ignore_index=True
).loc[18, 'Price']

1300000.0

## 37.3 Группировка данных в DataFrame

### МЕТОД GROUPBY()

→ by — задаёт имя или список имён столбцов, по которым производится группировка.

→ axis — задаёт ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.

→ as_index — задаёт, добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

In [40]:
# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

melb_df.groupby(by='Type', as_index=False).mean()
melb_df.groupby('Type')['Price'].mean()

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

In [ ]:
"""Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра 
Мельбурна.Для это го найдём минимальное значение расстояния от центра города до 
объекта в зависимости от его региона. Результат отсортируем по убыванию расстояния."""

melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

In [ ]:
# ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

"""Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём 
количество продаж, а также среднее и максимальное значения цен объектов недвижимости 
(Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем
по количеству продаж в порядке убывания:"""

melb_df.groupby('MonthSale')['Price'].agg(
    ['count', 'mean', 'min', 'max']
).sort_values(by='count', ascending=False)

В результате применения метода agg(), в который мы передали список с названиями 
интересующих нас агрегирующих функций, мы получаем DataFrame со столбцами 
count, mean и max, где для каждого месяца рассчитаны соответствующие параметры. 
Результат сортируем по столбцу count

In [ ]:
# полная информация обо всех основных статистических характеристиках внутри каждой группы
melb_df.groupby('MonthSale')['Price'].agg('describe')

In [ ]:
# метод agg() поддерживает использование и других функций. Передадим дополнительно 
# встроенную функцию set, чтобы получить множество из агентств недвижимости, которые
# работают в каждом из регионов
melb_df.groupby('Regionname')['SellerG'].agg(
    		['nunique', set]
)

In [ ]:
# Задание 37.3.1 Сгруппируйте данные по признаку количества комнат и найдите среднюю 
# цену объектов недвижимости в каждой группе.У квартир со сколькими комнатами 
# средняя цена наибольшая?
melb_df.groupby('Rooms')['Price'].mean().sort_values(ascending=False)

In [ ]:
# Задание 37.3.2 Какой регион имеет наименьшую протяжённость по географической 
# широте (Lattitude)?
melb_df.groupby('Regionname')['Lattitude'].std().sort_values()

In [ ]:
# Задание 37.3.3 Какая риелторская компания (SellerG) имеет наименьшую общую прибыль 
# за период с 1 мая по 1 сентября (включительно) 2017 года? Примечание.
# Не забудьте вести даты в формат datetime.

date1 = pd.to_datetime('2017-05-01')
date2 = pd.to_datetime('2017-09-01')
mask = (melb_df['Date'] >= date1) & (melb_df['Date'] <= date2)
melb_df[mask].groupby('SellerG')['Price'].sum().sort_values(ascending=True)

## 37.4 Сводные таблицы

In [64]:
# построить зависимость медианной цены и площади здания от числа комнат
melb_df.groupby(['Rooms'])[['Price', 'BuildingArea']].median()

,Price,BuildingArea
Rooms,,
1,385000.0,107.0
2,690000.0,126.0
3,950000.0,126.0
4,1285000.0,142.0
5,1660000.0,176.0
6,1800000.0,126.0
7,1496000.0,216.5
8,1515000.0,126.0
10,900000.0,126.0


In [67]:
# построить таблицу, в которой мы будем учитывать не только число комнат, но и 
# тип здания (Type). Для этого в параметрах метода groupby() укажем список из 
# нескольких интересующих нас столбцов. Чтобы финальный результат был представлен 
# в виде таблицы, к результату агрегирования применим метод unstack():
melb_df.groupby(['Rooms', 'Type'])['Price'].mean().unstack()

Type,house,townhouse,unit
Rooms,,,
1,8.668655e+05,5.927045e+05,3.899289e+05
2,1.017238e+06,7.101585e+05,6.104905e+05
3,1.109233e+06,9.847087e+05,8.505963e+05
4,1.462283e+06,1.217092e+06,1.037476e+06
5,1.877327e+06,1.035000e+06,NaN
6,1.869508e+06,NaN,5.200000e+05
7,1.920700e+06,NaN,NaN
8,1.510286e+06,NaN,2.250000e+06
10,9.000000e+05,NaN,NaN


Какие интересные выводы можно сделать из этой таблицы?
1	Пропуски в сводной таблице (NaN) говорят о том, что в наших данных нет соответствующих комбинаций признаков. Например, у нас нет информации о ценах на таунхаусы, где количество комнат больше пяти.
2	Наибольшей средней стоимостью (в 1,92 млн. австралийских долларов) обладают объекты типа house с семью жилыми комнатами. Наименьшая средняя стоимость — у однокомнатных домов типа unit (чуть меньше 400 тыс. австралийских долларов).
3	Сколько бы комнат ни было в доме, цена на объекты типа unit всегда ниже других (за исключением восьмикомнатных объектов).

### МЕТОД PIVOT_TABLE ДЛЯ ПОСТРОЕНИЯ СВОДНЫХ ТАБЛИЦ

### Основные параметры метода pivot_table()

→ values — имя столбца, по которому необходимо получить сводные данные, применяя агрегирующую функцию;

→ index — имя столбца, значения которого станут строками сводной таблицы;

→ columns — имя столбца, значения которого станут столбцами сводной таблицы;

→ aggfunc — имя или список имён агрегирующих функций (по умолчанию — подсчёт среднего, 'mean');

→ fill_value — значение, которым необходимо заполнить пропуски (по умолчанию пропуски не заполняются).

In [ ]:
melb_df.pivot_table(values='Price', index='Rooms', columns='Type').round(2)

In [ ]:
""" таблицу, в которой строками будут являться названия регионов (Regionname), 
а в столбцах будет располагаться наш «признак-мигалка» выходного дня (Weekend), 
который равен 1, если день был выходным, и 0 — в противном случае. В качестве 
значений сводной таблицы возьмём количество продаж."""
melb_df.pivot_table(
    values='Price',
    index='Regionname',
    columns='Weekend',
    aggfunc='count',
)

Выводы на основе результирующей таблицы

Число продаж резко возрастает в выходные вне зависимости от региона (приблизительно в 2-4 раза). То есть вероятность того, что дом продадут в выходные, гораздо выше вероятности, что его продадут в будни.
2	В отдалённых регионах (Victoria) коэффициент роста числа продаж выше, чем в центральных. Если в центральных регионах Metropolitan продажи по выходным в 2-2,5 раза выше, чем по будням, то в регионах Victoria число продаж в выходные вырастает в 3-4 раза.
Такой рост можно даже попытаться объяснить логически: в выходные дни у людей появляется свободное время, чтобы доехать до отдалённых пригородов с целью покупки дома.

In [79]:
"""Найдём, как зависит средняя и медианная площадь участка (Landsize) от типа 
объекта (Type) и его региона (Regionname). Чтобы посмотреть несколько статистических 
параметров, нужно передать в аргумент aggfunc список из агрегирующих функций. 
Построим такую сводную таблицу, где пропущенные значения заменим на 0:"""
pivot = melb_df.pivot_table(
    values='Landsize',
    index='Regionname',
    columns='Type',
    aggfunc=['median', 'mean'],
    fill_value=0
).round(0)
display(pivot)

median                   mean                 
Type                        house townhouse unit   house townhouse   unit
Regionname                                                               
Eastern Metropolitan        674.0     234.0  203   717.0     269.0  330.0
Eastern Victoria            843.0       0.0  230  3109.0       0.0  295.0
Northern Metropolitan       460.0     134.0    0   619.0     317.0  495.0
Northern Victoria           724.0       0.0    0  3355.0       0.0    0.0
South-Eastern Metropolitan  630.0     240.0  199   664.0     212.0  358.0
Southern Metropolitan       586.0     246.0    0   570.0     279.0  466.0
Western Metropolitan        531.0     198.0   62   508.0     245.0  558.0
Western Victoria            600.0       0.0    0   656.0       0.0    0.0

In [ ]:
melb_df['Quarter'] = melb_df['Date'].dt.quarter
melb_df['Quarter'].value_counts()
melb_df[melb_df['Quarter']  == 2]['Price'].count()


4359

### МНОГОМЕРНЫЕ СВОДНЫЕ ТАБЛИЦЫ

In [ ]:
""" построим таблицу, в которой по индексам будут располагаться признаки метода 
продажи (Method) и типа объекта (Type), по столбцам — наименование региона (Regionname), 
а на пересечении строк и столбцов будет стоять медианная цена объекта (Price)."""

melb_df.pivot_table(
    values='Price',
    index=['Method','Type'],
    columns='Regionname',
    aggfunc='median',
    fill_value=0
)

### ДОСТУП К ДАННЫМ В СВОДНОЙ ТАБЛИЦЕ

In [ ]:
# Возьмём сводную таблицу pivot, которую мы создавали ранее:
# Выведем её столбцы с помощью атрибута columns.
pivot.columns

# из таблицы pivot мы можем получить средние значения площадей участков для т
# ипа здания unit, просто последовательно обратившись по имени столбцов
display(pivot['mean']['unit'])

In [ ]:
# регионы, в которых средняя площадь здания для домов типа house меньше их медианной 
# площади, то мы можем найти их следующим образом:
mask = pivot['mean']['house'] < pivot['median']['house']
filtered_pivot = pivot[mask]
display(filtered_pivot)

In [81]:
# получить индексы отфильтрованной таблицы, можно воспользоваться атрибутом index 
# и обернуть результат в список.
print(list(filtered_pivot.index))

['Southern Metropolitan', 'Western Metropolitan']


In [82]:
"""Задание 37.4.2 Составьте сводную таблицу, которая показывает зависимость 
медианной площади (BuildingArea) здания от типа объекта недвижимости (Type) 
и количества жилых комнат в доме (Rooms).Для какой комбинации признаков площадь 
здания наибольшая?"""
pivot = melb_df.pivot_table(
    values='BuildingArea',
    index='Type',
    columns='Rooms',
    aggfunc='median',
    fill_value=0
)
display(pivot)

Rooms,1,2,3,4,5,6,7,8,10
Type,,,,,,,,,
house,126.0,126,126,141.0,177,126,216.5,126,126
townhouse,88.0,114,126,159.5,152,0,0.0,0,0
unit,69.5,110,126,126.0,0,171,0.0,126,0


In [86]:
"""Задание 37.4.3 Составьте сводную таблицу, которая показывает зависимость медианной 
цены объекта недвижимости (Price) от риелторского агентства (SellerG) и типа здания
(Type). У какого агентства (во вновь созданной таблице) максимальная медианная цена 
для зданий типа unit?"""
pivot = melb_df.pivot_table(
    values='Price',
    index='SellerG',
    columns='Type',
    aggfunc='median',
)
#print(pivot)
max_unit_price = pivot['unit'].max()
print(pivot[pivot['unit'] == max_unit_price].index[0])

Nick


## 37.5 Объединение DataFrame: Знакомство с новыми данными

Надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах.

Вот как мы будем действовать:
1. Склеим таблицы ratings1 и ratings2 в единую структуру.
2. К полученной таблице с рейтингами присоединим столбец с датой выставления рейтинга.
3. Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

In [104]:
ratings1 = pd.read_csv('data/ratings1.csv')
ratings2 = pd.read_csv('data/ratings2.csv')
dates = pd.read_csv('data/dates.csv')
movies = pd.read_csv('data/movies.csv')


In [ ]:
ratings1.head()
ratings2.head()
dates.head()
movies.head()

ratings1.info()
ratings2.info()
dates.info()
movies.info()

In [100]:
# Сколько уникальных фильмов представлено в таблице movies?
movies['movieId'].nunique()

9742

In [99]:
#Сколько уникальных пользователей в таблице ratings1?
ratings1['userId'].nunique()

274

In [124]:
# В каком году было выставлено больше всего оценок?
dates['Year'] = pd.to_datetime(dates['date']).dt.year
dates['Year'].value_counts()
dates.drop('Year', axis=1, inplace=True)


## 37.6 Объединение DataFrame: concat()

Основные параметры метода concat()

→ objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;

→ axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;

→ join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;

→ ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в соответствующем порядке.

In [ ]:
"""склеим таблицы ratings1 и ratings2. Для этого передадим их в списке в функцию 
concat(). По умолчанию объединение происходит по строкам, что нам и нужно, поэтому 
параметр axis мы не трогаем."""

ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

In [114]:
"""знаем количество строк в таблицах ratings и dates, ведь нам предстоит 
объединять их по столбцам:"""

print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True.

In [126]:
# УДАЛЕНИЕ ДУБЛЕЙ
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [ ]:
"""можем добавить к нашей таблице с оценками даты их выставления. Для этого 
конкатенируем таблицы ratings и dates по столбцам."""

ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

In [ ]:
# Задание 37.6.3 НЕ ЗАПУСКАТЬ!
import pandas as pd
import os

def concat_users_files(path):
    """
    Вам необходимо написать функцию concat_user_files(path), параметром которой является path - путь до директории. 
    Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. 
    Не забудьте обновить индексы результирующей таблицы после объединения.
    Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 
    """
    data = pd.DataFrame()
    file_names = os.listdir(path)
    file_names.sort()
    for file in file_names:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data], axis=0, ignore_index=True)
    data = data.drop_duplicates()
    return data

if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

## 37.7 Объединение DataFrame: join(), merge()

In [7]:
from IPython.display import Image

![Image](https://lms.skillfactory.ru/assets/courseware/v1/7f8be8485301f840fe8bc2a8f6cf444f/asset-v1:SkillFactory+DAPR-2.0+MAR2021+type@asset+block/dst3-u1-md12_7_10.png)

### Основные параметры метода join()

→ other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».

→ how — параметр типа объединения. Может принимать значения: 'inner', 'left' (left outer), 'right' (right outer) и 'outer' (full outer). По умолчанию параметр установлен на 'left'.

→ on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».

→ lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

In [ ]:
# объединим таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, 
# установим один из суффиксов, чтобы избежать ошибки:

joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

In [ ]:
"""Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо 
использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать 
с помощью метода set_index(). Также необходимо указать название ключа в параметре on."""

joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

### Основные параметры метода merge()

→ right — присоединяемая таблица. По умолчанию она является «правой».

→ how — параметр типа объединения. По умолчанию принимает значение 'inner'.

→ on — параметр, который определяет, по какому столбцу происходит объединение. Определяется автоматически, но рекомендуется указывать вручную.

→ left_on — если названия столбцов в «левой» и «правой» таблицах не совпадают, то данный параметр отвечает за наименования ключевого столбца исходной таблицы.

→ right_on — аналогично предыдущему, параметр отвечает за наименование ключевого столбца присоединяемой таблицы.

→ lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

In [ ]:
merged = ratings_dates.merge(
    movies,
    on='movieId',
    how='left'
)
display(merged.head())

In [135]:
"""Осталось только проверить, что число строк в таблице ratings_dates совпадает с 
числом строк в результирующей таблице merged."""

print('Число строк в таблице ratings_dates: ', ratings_dates.shape[0])
print('Число строк в таблице merged: ', merged.shape[0])
print(ratings_dates.shape[0] == merged.shape[0])

Число строк в таблице ratings_dates:  100836
Число строк в таблице merged:  100836
True


In [ ]:
"""Рассмотрим пример: объединим таблицы ratings1 и ratings2, как мы уже делали 
раньше, но теперь используем метод merge():"""

merge_ratings = ratings1.merge(ratings2, how='outer')

print('Число строк в таблице merge_ratings: ', merge_ratings.shape[0])
display(merge_ratings)

In [153]:
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

display(items_df)
display(purchase_df)

,item_id,vendor,stock_count
0,417283,Samsung,54
1,849734,LG,33
2,132223,Apple,122
3,573943,Apple,18
4,19475,LG,102
5,3294095,Apple,43
6,382043,Samsung,77
7,302948,Samsung,143
8,100132,LG,60
9,312394,ZTE,19


,purchase_id,item_id,price
0,101,417283,13900
1,101,849734,5330
2,101,132223,38200
3,112,573943,49990
4,121,19475,9890
5,145,3294095,33000
6,145,382043,67500
7,145,302948,34500
8,145,103845,89900
9,221,100132,11400


Информация в таблицах представлена в виде следующих столбцов:

item_id — идентификатор модели;
vendor — производитель модели;
stock_count — имеющееся на складе количество данных моделей (в штуках);
purchase_id — идентификатор покупки;
price — стоимость модели в покупке.

In [ ]:
"""Вам необходимо сделать следующее:
1) Сформируйте DataFrame merged, так чтобы после объединения purchase_df и 
items_df остались модели, которые учтены на складе и имели продажи.
2) На основе таблицы merged найдите суммарную выручку, которую можно было бы получить 
от продажи всех товаров, имеющихся на складе. Результат занесите в переменную income."""

merged = items_df.merge(purchase_df, how='inner', on='item_id')
merged['total'] = merged['stock_count'] * merged['price'] 
income = merged['total'].sum()

## 37.8 Закрепление знаний

In [159]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [ ]:
joined = pd.read_csv('data/ratings_movies.csv')
display(joined.info())
joined.head()

In [ ]:
melb_df['Quarter'] = melb_df['Date'].dt.quarter
melb_df['Quarter'].value_counts()
melb_df[melb_df['Quarter']  == 2]['Price'].count()


4359

In [ ]:
"""Задание 37.8.1
Создайте в таблице новый признак year_release, который соответствует году выпуска 
фильма. У скольких фильмов не указан год их выпуска?"""

joined['year_release'] = joined['title'].apply(get_year_release)
joined.info()

In [ ]:
"""Задание 37.8.2
Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?"""

mask = joined['year_release'] == 1999
joined[mask].groupby('title')['rating'].mean().sort_values()

In [ ]:
"""Задание 37.8.3
Какое сочетание жанров фильмов, выпущенных в 2010 году, получило наименьшую 
среднюю оценку?"""

mask = joined['year_release'] == 2010
joined[mask].groupby('genres')['rating'].mean().sort_values()

In [ ]:
"""Задание 37.8.4
Какой пользователь посмотрел фильмы наибольшего количества жанров?"""

joined.groupby('userId')['genres'].nunique().sort_values(ascending=False)

In [ ]:
"""Задание 37.8.5
Найдите пользователя, который выставил наименьшее количество оценок, но его средняя 
оценка фильмам наибольшая."""

joined.groupby('userId')['rating'].agg(
    ['count', 'mean']
).sort_values(['count', 'mean'], ascending=[True, False])

In [ ]:
"""Задание 37.8.6
Найдите сочетание жанров фильма, выпущенного в 2018 году, который имеет наибольший 
средний рейтинг, и при этом число его просмотров больше 10."""

mask = joined['year_release'] == 2018
grouped = joined[mask].groupby('genres')['rating'].agg(
    ['mean', 'count']
)
grouped[grouped['count']>10].sort_values(
    by=['mean', 'count'],
    ascending=[False, False]
)

In [ ]:
"""Задание 37.8.7
Добавьте в таблицу новый признак year_rating — год выставления оценки. Создайте сводную 
таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления 
оценки и жанра. Выберите верные варианты ответа, исходя из построенной таблицы."""

joined['date'] = pd.to_datetime(joined['date'])
joined['year_rating'] = joined['date'].dt.year
pivot = joined.pivot_table(
    index='year_rating',
    columns='genres',
    values='rating',
    aggfunc='mean'
)
display(pivot)

Порядок дальнейших действий такой:

Прочитайте данные файлы, посмотрите на содержимое таблиц и проанализируйте его.
Подумайте, как связаны данные таблицы (какими ключевыми столбцами).
Объедините заданные таблицы в таблицу orders_products, чтобы в результирующей таблице оказалась информация обо всех заказах, но не оказалось информации о продуктах, на которых заказов ещё не поступало. 
Ответьте на несколько вопросов по таблице orders_products.

In [ ]:
orders = pd.read_csv('data/orders.csv', sep=';') # данные о заказах
display(orders.info())
orders.head()

In [ ]:
products = pd.read_csv('data/products.csv', sep=';') # данные о товарах
display(products.info())
products.head()

In [ ]:
orders_products = orders.merge(
    products, 
    how='left', 
    left_on='ID товара',
    right_on='Product_ID'
)
display(orders_products)

In [198]:
"""Задание 37.8.8
Какой идентификатор (Order ID) имеет заказ, для которого не оказалось информации 
о товаре?"""

orders_products.tail(1)['Order ID']

17    0
Name: Order ID, dtype: int64

In [199]:
"""Задание 37.8.9
На какой товар была произведена отмена?"""

orders_products[orders_products['Отменен'] == 'Да']['Name']

6    Носки беговые Camino
Name: Name, dtype: object

In [200]:
"""Задание 37.8.10
Какой покупатель принёс наибольшую суммарную прибыль интернет-магазину 
за указанный период?"""

orders_products.groupby('ID Покупателя')['Price'].sum().sort_values(ascending=False)

ID Покупателя
7     17096.0
5     14042.0
8       697.0
1       448.0
9       344.0
10      199.0
Name: Price, dtype: float64